# Baseten custom model example deployment

<a href="https://colab.research.google.com/github/basetenlabs/demos/blob/main/deployment/baseten_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install sklearn baseten

In [ ]:
# Model training

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

iris = load_iris()
data_x = iris['data']
data_y = iris['target']

model = RandomForestClassifier()
model.fit(data_x, data_y)

In [ ]:
# Create a Truss

import truss

truss.init("rfc_truss")

In [ ]:
# Generate model files

import joblib

# Pickled model
joblib.dump(model, 'rfc_truss/data/rfc_model.pkl', compress=True)

# Model class
MODEL_CODE_AS_STR = """
import joblib
from typing import Dict, List

class Model:
    def load(self):
        self.model = joblib.load(open('data/rfc_model.pkl', 'rb'))

    def predict(self, inputs):
        output = self.model.predict(inputs["inputs"])
        response = {"predictions" : output }
        return response
"""

with open("rfc_truss/model/model.py", "w") as py_file:
    py_file.write(MODEL_CODE_AS_STR)

In [ ]:
# Add dependencies

tr = truss.from_directory("rfc_truss")
tr.add_python_requirement("joblib==1.1.0")
tr.add_python_requirement("scikit-learn==1.0.2")

In [ ]:
# Model deployment

import baseten

api_key = "PASTE API KEY HERE"
baseten.login(api_key)

baseten.deploy_truss(
    tr,
    model_name="Iris RFC Model (custom deployment)"
)

In [ ]:
# After the deployment is finished, call your new model!

deployed_model_id = "PASTE VERSION ID HERE" # See deployed model page to find version ID
model_input = [[0, 0, 0, 0]]

deployed_model = baseten.deployed_model_version_id(deployed_model_id)
deployed_model.predict(model_input)

In [ ]:
# Or call your model as an API

import requests
import json

endpoint = f"https://app.baseten.co/model_versions/{deployed_model_id}/predict"
auth = {"Authorization": f"Api-Key {api_key}"}
data = json.dumps({"inputs": model_input})

resp = requests.post(
    endpoint,
    headers=auth,
    data=data
)

resp.json()